In [ ]:
# ----------------------------------------------------------------------------------
# PRIMER MODELO DE PARSER POR VOZ (Google Cloud Speech-to-Text)
# ESCENARIO / PRUEBA: "Prueba 1 – Parser sintáctico básico para comandos de junta"
#
# DESCRIPCIÓN GENERAL
# - Captura audio del micrófono en tiempo real (16 kHz, mono, LINEAR16) y lo envía
#   por streaming a Google Cloud Speech-to-Text (idioma: es-ES) con resultados
#   intermedios y definitivos.
# - Al recibir una hipótesis FINAL, intenta interpretar comandos del tipo:
#       "junta N a ±M grados"
# - El parser normaliza texto (minúsculas, sin tildes), convierte "a más/menos X"
#   en signos (+/-) y acepta coma decimal ("12,5").
# - Cuando el comando es válido, publica una línea simulada de salida serial:
#       [SERIAL_SIM] J<junta>:<ángulo>
# - Usar spaCy para validar entidades/pos/dependencias y robustecer el parser.
# - Aceptar múltiples comandos en una sola frase ("junta 1 a 10 y junta 2 a -5").

import os
import queue
import time
import re
import sounddevice as sd
from google.cloud import speech
from google.oauth2 import service_account
from six.moves import queue as six_queue
import unidecode
import spacy

nlp = spacy.load("es_core_news_sm")

#  "junta N a ±M grados" 
def parse_command(text: str):

    # 1) Normalizar: quitar tildes, pasar a minúsculas
    t = unidecode.unidecode(text).lower()
    # 2) Transformar "a mas X"→"+X" y "a menos X"→"-X"
    t = re.sub(r"\ba\s+mas\s+(\d+)", r"+\1", t)
    t = re.sub(r"\ba\s+menos\s+(\d+)", r"-\1", t)
    # 3) Buscar patrón "junta <n> ... <±m> grados"
    m = re.search(r"junta\s+(\d+).*?([+-]?\d+[.,]?\d*)\s*grados", t)
    if not m:
        return None, None
    return int(m.group(1)), float(m.group(2).replace(",", "."))

# CREDENCIALES y CLIENTE de Google Speech-to-Text 
KEYFILE = "STT_demo.json"  
creds   = service_account.Credentials.from_service_account_file(KEYFILE)
client  = speech.SpeechClient(credentials=creds)

#  PARÁMETROS de AUDIO 
RATE  = 16000             # Hz
CHUNK = int(RATE / 10)    # 100 ms (1600 muestras)

# CONFIGURACIÓN de Google STT 
recognition_config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=RATE,
    language_code="es-ES",               # Español de España
    enable_automatic_punctuation=True    # añade comas y puntos 
)
streaming_config = speech.StreamingRecognitionConfig(
    config=recognition_config,
    interim_results=True                 # resultados parciales en vivo
)

# ─── COLA de AUDIO para el STREAMING ───────────────────────────────────────
audio_q: six_queue.Queue[bytes] = queue.Queue()

def sd_callback(indata, frames, time_info, status):
    if status:
        print(f"[SD status] {status}", flush=True)
    try:
        data_bytes = indata.tobytes()
    except AttributeError:
        data_bytes = bytes(indata)
    audio_q.put(data_bytes)

def audio_generator():

    while True:
        chunk = audio_q.get()
        if chunk is None:
            return
        yield speech.StreamingRecognizeRequest(audio_content=chunk)

#  BUCLE de IMPRESIÓN y PARSING de RESPUESTAS 
def listen_print_loop(responses):

    for resp in responses:
        if not resp.results:
            continue
        result = resp.results[0]
        txt    = result.alternatives[0].transcript.strip()

        if result.is_final:
            # Transcripción definitiva
            print(f"\r[FIN]  {txt}{' '*20}")
            # Parsear comando de junta
            joint, angle = parse_command(txt)
            if joint is not None:
                print(f"[SERIAL_SIM] J{joint}:{angle}")
        else:
            # Transcripción parcial
            print(f"\r[LIVE] {txt}", end="", flush=True)

# FUNCIÓN PRINCIPAL 
def main():
    print("Transcribiendo...")
    while True:
        try:
            # Abrir flujo de audio y arrancar STT
            with sd.RawInputStream(
                samplerate=RATE,
                blocksize=CHUNK,
                dtype="int16",
                channels=1,
                callback=sd_callback
            ):
                requests  = audio_generator()
                responses = client.streaming_recognize(streaming_config, requests)
                listen_print_loop(responses)
        except Exception as e:
            print(f"\n[ERROR stream] {e}\nReintentando en 1 s…")
            time.sleep(1)
        else:
            break

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        audio_q.put(None)
        print("\nTerminando…")


Transcribiendo...
[FIN]  mueve la Junta 340 grados                    
[SERIAL_SIM] J34:0.0
[FIN]  9 la Junta 4 60 grados                    
[SERIAL_SIM] J4:60.0
[FIN]  necesito que muevas la Junta 360 grados.                    
[SERIAL_SIM] J36:0.0
[FIN]  Necesito que muevas la Junta 3 a 60 grados.                    
[SERIAL_SIM] J3:60.0

Terminando…


In [ ]:
#-------------------------------------------------------------------------------
# MODELO DE PARSER SEMÁNTICO (Google Cloud Speech-to-Text + spaCy)
# -----------------------Prueba 2 -------------------------------------------------
#
# DESCRIPCIÓN GENERAL
# - Captura audio del micrófono y lo envía en streaming a Google Cloud STT.
# - Reconoce múltiples formas de dar órdenes a juntas o eslabones, con soporte
#   de sinónimos y ordinales (ej. “junta segunda”, “segundo eslabón”, “link 3”).
# - Parser implementado con combinación de expresiones regulares y diccionario
#   de palabras ordinales → convierte texto natural en IDs de juntas numéricas.
# - Extrae todos los comandos presentes en una misma frase, devolviendo una lista
#   de pares (joint_id, angle).
#   Ejemplo:
#       “mueve la segunda junta a 40 grados y la cuarta a menos 30” 
#       ⇢ [(2, 40), (4, -30)]
# - Ordinales soportados: primero → 1, segundo → 2, … décimo → 10 (masculino y femenino).   
# - Detección y corrección de errores comunes de parsing (“37 0” interpretado como
#   junta=3, ángulo=70).
# - Ángulos aceptan signo explícito o palabras (“+”, “-”, “más”, “menos”), y coma decimal.

# LIMITACIONES
# - Actualmente el parser depende de regex y diccionario; spaCy aún no se aprovecha
#   para dependencias gramaticales o contexto más amplio.
# - Integración con control físico de robot vía comunicación serie/UDP en lugar

import os
import queue
import time
import re
import sounddevice as sd
from google.cloud import speech
from google.oauth2 import service_account
from six.moves import queue as six_queue
import unidecode
import spacy

MAX_JOINT = 10          
# Cargar el modelo 
nlp = spacy.load("es_core_news_sm")

# SINÓNIMOS Y ORDINALES
JOINT_SYNONYMS = {
    "junta", "eslabon", "eslabón", "conexion", "conexión",
    "articulacion", "articulación", "link", "Posicion", "Posición"
}

ORDINALS = {
    "primer": 1, "primero": 1, "primera": 1,
    "segundo": 2, "segunda": 2,
    "tercero": 3, "tercera": 3,
    "cuarto": 4, "cuarta": 4,
    "quinto": 5, "quinta": 5,
    "sexto": 6, "sexta": 6,
    "septimo": 7, "septima": 7, "séptimo": 7, "séptima": 7,
    "octavo": 8, "octava": 8,
    "noveno": 9, "novena": 9,
    "decimo": 10, "decima": 10, "décimo": 10, "décima": 10
}

# ─── PARSER basado en spaCy + regex ────────────────────────────────────
# ─── PARSER basado en spaCy + regex ────────────────────────────────────
def parse_commands(text: str):
    """
    Devuelve una lista de tuplas (joint_id, angle) encontradas en `text`.

    Ahora entiende tanto:
        «junta segunda a 40 grados»  ──⇢ (2, 40)
        «segunda junta a 40 grados»  ──⇢ (2, 40)
    """
    norm = unidecode.unidecode(text).lower()

    # Patrón común
    SYN   = r"(?:junta|eslabon|eslabón|articulacion|articulación|" \
            r"conexion|conexión|link|posicion|posición)"
    NUM   = r"(?P<joint>\d+|[a-záéíóú]+)"          # nº cardinal u ordinal
    SIGN  = r"(?P<sign>[+\-]|mas|menos)?"
    ANG   = r"(?P<ang>\d+[.,]?\d*)"               # número del ángulo

    # 1) “junta 4 …”  |  “junta cuarta …”
    patt1 = re.compile(
        rf"{SYN}\s+{NUM}\s*(?:a\s*)?{SIGN}\s*{ANG}",
        re.IGNORECASE,
    )
    # 2) “cuarta junta …”  |  “4ª junta …”
    patt2 = re.compile(
        rf"{NUM}\s+{SYN}\s*(?:a\s*)?{SIGN}\s*{ANG}",
        re.IGNORECASE,
    )

    results = []          # [(joint, angle)]       en orden de aparición
    for m in sorted(
        list(patt1.finditer(norm)) + list(patt2.finditer(norm)),
        key=lambda x: x.start()
    ):
        raw_joint = m.group("joint")
        joint = int(raw_joint) if raw_joint.isdigit() else ORDINALS.get(raw_joint)
        if joint is None:
            continue                                # ordinal no reconocido

        angle = float(m.group("ang").replace(",", "."))
        if m.group("sign") in {"-", "menos"}:
            angle = -angle

        # --- corrección del bug “37 0” … (igual que antes) --------------
        if angle == 0 and joint > MAX_JOINT and joint < 100:
            s          = str(joint)
            new_joint  = int(s[:-1])
            new_angle  = int(s[-1]) * 10
            if 1 <= new_joint <= MAX_JOINT and 0 < new_angle <= 180:
                joint, angle = new_joint, new_angle

        results.append((joint, angle))

    return results


# CREDENCIALES 
KEYFILE = "STT_demo.json"
creds   = service_account.Credentials.from_service_account_file(KEYFILE)
client  = speech.SpeechClient(credentials=creds)

# PARÁMETROS de AUDIO
RATE  = 16000             # Hz
CHUNK = int(RATE / 10)    # 100 ms

# CONFIGURACIÓN STT 
recognition_config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=RATE,
    language_code="es-ES",
    enable_automatic_punctuation=True
)
streaming_config = speech.StreamingRecognitionConfig(
    config=recognition_config,
    interim_results=True
)

# COLA de AUDIO y CALLBACK
audio_q: six_queue.Queue[bytes] = queue.Queue()

def sd_callback(indata, frames, time_info, status):
    if status:
        print(f"[SD status] {status}", flush=True)
    data_bytes = indata.tobytes() if hasattr(indata, "tobytes") else bytes(indata)
    audio_q.put(data_bytes)

def audio_generator():
    while True:
        chunk = audio_q.get()
        if chunk is None:
            return
        yield speech.StreamingRecognizeRequest(audio_content=chunk)

#  BUCLE de TRANSCRIPCIÓN y PARSING 
def listen_print_loop(responses):
    for resp in responses:
        if not resp.results:
            continue
        result = resp.results[0]
        txt = result.alternatives[0].transcript.strip()

        if result.is_final:
            print(f"\r[FIN]  {txt}{' '*20}")

            pairs = parse_commands(txt)            # lista de (junta, ángulo)

            if pairs:
                print(f"[Acciones] → {pairs}")      # debug compacto
                for joint, angle in pairs:         # envías cada comando
                    print(f"[SERIAL_SIM] J{joint}:{angle}")
            else:
                print("Sin ordenes")

        else:
            print(f"\r[LIVE] {txt}", end="", flush=True)


# ─── FUNCIÓN PRINCIPAL ─────────────────────────────────────────────────
def main():
    print("Transcribiendo")
    while True:
        try:
            with sd.RawInputStream(
                samplerate=RATE,
                blocksize=CHUNK,
                dtype="int16",
                channels=1,
                callback=sd_callback
            ):
                requests  = audio_generator()
                responses = client.streaming_recognize(streaming_config, requests)
                listen_print_loop(responses)
        except KeyboardInterrupt:
            audio_q.put(None)
            print("\nTerminando…")
            break
        except Exception as e:
            print(f"\n[ERROR stream] {e}\nReintentando en 1 s…")
            time.sleep(1)

if __name__ == "__main__":
    main()


Transcribiendo
[FIN]  Buenas hola hola hola hola.                    
Sin ordenes
[FIN]  Quiero que me muevas la Junta 3 30 grados a la derecha.                    
[Acciones] → [(3, 30.0)]
[SERIAL_SIM] J3:30.0
[FIN]  Quiero que mueva la punta 2 30 grados a la izquierda y la Junta 330.                    
[Acciones] → [(3, 30)]
[SERIAL_SIM] J3:30
[FIN]  Quiero que muevas la Junta 230 grados a la izquierda y la Junta 460 grados.                    
[Acciones] → [(2, 30), (4, 60)]
[SERIAL_SIM] J2:30
[SERIAL_SIM] J4:60

Terminando…


In [ ]:
# --------------------------------------------------------------------------------
# -------MODELO DE PARSER SEMÁNTICO AVANZADO (Google Cloud STT + spaCy)-----------
# ------------Prueba 3 ---------------------------------------------------------
#
# DESCRIPCIÓN GENERAL
# - Captura audio en tiempo real (16 kHz) y lo envía a Google Cloud Speech-to-Text.
# - Reconoce frases naturales con verbos de movimiento (mover, girar, rotar, colocar…),
#   sinónimos de “junta” (eslabón, articulación, link, posición, etc.) y ordinales
#   (primero, segunda, décima, etc.).
# - Permite dividir órdenes largas en sub-frases (“y”, “luego”, “después”, comas).
# - Extrae todos los comandos presentes y devuelve una lista única de pares
#   (joint_id, angle) en orden de aparición.
#   Ejemplo:
#       “mueve la segunda junta a 40 grados, luego gira la cuarta a menos 30”
#       ⇢ [(2, 40), (4, -30)]
# - Aprovechar el análisis morfosintáctico de spaCy para tolerar frases aún más libres.
# - Añadir reconocimiento de series de comandos encadenados con tiempos (“después de 2 segundos”).
# - Conectar con hardware real vía pyserial/ROS en lugar de simulación por consola.




import re
import unidecode
import spacy
import os
import time
import sounddevice as sd
from google.cloud import speech
from google.oauth2 import service_account
from six.moves import queue as six_queue


nlp = spacy.load("es_core_news_sm")

# Lista de lemas de “mover/rotar/girar/etc.” que queremos reconocer
MOVE_VERBS = [
    "mover", "girar", "rotar", "colocar", "posicionar",
    "rote", "rota", "gira", "mueve", "move"
]

# Sinónimos de “junta”
JNT_SYNONYMS = [
    "junta", "eslabon", "eslabón", "articulacion", "articulación",
    "conexión", "conexion", "link", "posición", "posicion"
]
JNT = r"(?:%s)" % "|".join(JNT_SYNONYMS)

# Mapeo de ordinales de texto → número
ORDINALS_MAP = {
    "primer":   1, "primero":   1, "primera":   1,
    "segundo":  2, "segunda":   2,
    "tercero":  3, "tercera":   3,
    "cuarto":   4, "cuarta":    4,
    "quinto":   5, "quinta":    5,
    "sexto":    6, "sexta":     6,
    "séptimo":  7, "septima":   7, "séptima":  7, "septimo": 7,
    "octavo":   8, "octava":    8,
    "noveno":   9, "novena":    9,
    "décimo":  10, "decimo":   10, "décima": 10, "decima": 10,
}

MAX_JOINT = 10   # Número máximo de juntas soportadas (ajusta según tu robot)
                


def split_clauses(text: str):
    """
    Divide el texto en clausulas cortas, separadas por comas, " y ", " luego ",
    " después ", " por último ". Devuelve lista de subfrases limpiadas y en minúsculas.
    """

    low = unidecode.unidecode(text.lower())
    # Divido cada vez que hay una coma, " y ", " luego ", " después ", " por último "
    trozos = re.split(r',|\sluego\s|\sy\s|\sdespués\s|\spor último\s', low)
    return [t.strip() for t in trozos if t.strip()]


def parse_all_commands(text: str):
    """
    Devuelve una lista de tuplas (joint_id, angle) encontradas en `text`,
    incluyendo signos escritos (“menos”/“mas”).
    """
    # Normalizar texto
    norm = unidecode.unidecode(text.lower())
    # Quito posibles comas
    norm = re.sub(r',\s*(?=\d)', ' a ', norm)


    # Dividir en cláusulas
    clauses = split_clauses(norm)
    results = []

    # Definir patrón de signo y número
    SIGN   = r"(?P<sign>[+\-]|menos|mas)?\s*"       # captura “-”, “+”, “menos” o “mas”
    NUM    = r"(?P<ang>\d+(?:[.,]\d+)?)"           # captura número con coma/punto opcional
    grados = r"(?:\s*grados?)?"                    # “grados” opcional

    # Patrones con signo incluido antes del número
    pat_a = re.compile(
        rf"(?:{'|'.join(MOVE_VERBS)})\s+(?:la\s+)?"
        rf"(?P<ord>[a-záéíóú]+)\s+{JNT}(?:\s+a)?\s*"
        rf"{SIGN}{NUM}{grados}",
        re.IGNORECASE
    )
    pat_b = re.compile(
        f"(?:{'|'.join(MOVE_VERBS)})\s+{JNT}\s+"
        rf"(?P<ord>[a-záéíóú]+)(?:\s+a)?\s*"
        rf"{SIGN}{NUM}{grados}",
        re.IGNORECASE
    )
    pat_c = re.compile(
        rf"(?P<ord>[a-záéíóú]+)\s+(?:{JNT}\s*)?"
        rf"{SIGN}{NUM}{grados}",
        re.IGNORECASE
    )
    pat_d = re.compile(
        rf"{JNT}\s+(?P<num>\d+)\s+(?:a\s*)?"
        rf"{SIGN}{NUM}{grados}",
        re.IGNORECASE
    )
    pat_e = re.compile(
        rf"(?P<num>\d+)\s+(?:a\s*)?"
        rf"{SIGN}{NUM}{grados}",
        re.IGNORECASE
    )

    for cl in clauses:
        for pat in (pat_a, pat_b, pat_c):
            for m in pat.finditer(cl):
                ord_literal = m.group("ord")
                joint = ORDINALS_MAP.get(ord_literal)
                if not joint:
                    continue
                raw = m.group("ang").replace(",", ".")
                angle = float(raw)
                sign = m.group("sign")
                if sign in ("-", "menos"):
                    angle = -angle
                results.append((joint, angle))

        # Opción D: “junta 3 a -30 grados”
        for m in pat_d.finditer(cl):
            j = int(m.group("num"))
            if not (1 <= j <= MAX_JOINT):
                continue
            raw = m.group("ang").replace(",", ".")
            angle = float(raw)
            sign = m.group("sign")
            if sign in ("-", "menos"):
                angle = -angle
            results.append((j, angle))

        # Opción E: “3 a -40 grados” (sin la palabra “junta”)
        for m in pat_e.finditer(cl):
            start = m.start()
            previo = cl[max(0, start-6):start].strip()
            if previo.endswith("junta") or previo.endswith("juntas"):
                continue
            j = int(m.group("num"))
            if not (1 <= j <= MAX_JOINT):
                continue
            raw = m.group("ang").replace(",", ".")
            angle = float(raw)
            sign = m.group("sign")
            if sign in ("-", "menos"):
                angle = -angle
            results.append((j, angle))

    # Eliminar duplicados manteniendo orden
    seen = set()
    uniq = []
    for pair in results:
        if pair not in seen:
            seen.add(pair)
            uniq.append(pair)

    return uniq



KEYFILE = "STT_demo.json"
creds   = service_account.Credentials.from_service_account_file(KEYFILE)
client  = speech.SpeechClient(credentials=creds)

RATE  = 16000             # Hz
CHUNK = int(RATE / 10)    # 100 ms

recognition_config = speech.RecognitionConfig(
    encoding                   = speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz          = RATE,
    language_code              = "es-ES",
    enable_automatic_punctuation = True
)
streaming_config = speech.StreamingRecognitionConfig(
    config         = recognition_config,
    interim_results = True
)

audio_q: six_queue.Queue[bytes] = queue.Queue()

def sd_callback(indata, frames, time_info, status):
    if status:
        print(f"[SD status] {status}", flush=True)
    data_bytes = indata.tobytes() if hasattr(indata, "tobytes") else bytes(indata)
    audio_q.put(data_bytes)

def audio_generator():
    while True:
        chunk = audio_q.get()
        if chunk is None:
            return
        yield speech.StreamingRecognizeRequest(audio_content=chunk)

def listen_print_loop(responses):
    for resp in responses:
        if not resp.results:
            continue
        result = resp.results[0]
        txt = result.alternatives[0].transcript.strip()

        if result.is_final:
            print(f"\r[FIN]  {txt}{' '*20}")

            pairs = parse_all_commands(txt)
            if pairs:
                print(f"[Entendí] → {pairs}")
                for j, a in pairs:
                    print(f"[SERIAL_SIM] J{j}:{a}")
            else:
                print("[Entendí] → (sin órdenes detectadas)")
        else:
            print(f"\r[LIVE] {txt}", end="", flush=True)

def main():
    print("Transcribiendo")
    while True:
        try:
            with sd.RawInputStream(
                samplerate = RATE,
                blocksize  = CHUNK,
                dtype      = "int16",
                channels   = 1,
                callback   = sd_callback
            ):
                requests  = audio_generator()
                responses = client.streaming_recognize(streaming_config, requests)
                listen_print_loop(responses)
        except KeyboardInterrupt:
            audio_q.put(None)
            print("\nTerminando…")
            break
        except Exception as e:
            print(f"\n[ERROR stream] {e}\nReintentando en 1 s…")
            time.sleep(1)

if __name__ == "__main__":
    main()


Transcribiendo
[FIN]  Quiero que primero muevas la Junta 465.3 grados y después de eso logres mover la Junta 5, 40 grados y coloques la Junta 1 a 30 grados.                    
[Entendí] → [(5, 40.0), (1, 30.0)]
[SERIAL_SIM] J5:40.0
[SERIAL_SIM] J1:30.0
[FIN]  Primero quiero que muevas la Junta 5 a 65.3 grados y después de eso muevas la Junta 3 a 40 grados y por último muevas la Junta 3 otra vez, pero menos 30 grados.                    
[Entendí] → [(5, 65.3), (3, 40.0)]
[SERIAL_SIM] J5:65.3
[SERIAL_SIM] J3:40.0

Terminando…
